# Initial imports

In [ ]:
import pandas as pd

import os
import shutil
import torch
from typing import Dict

from pathlib import Path
import matplotlib.pyplot as plt

from time import perf_counter

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
CACHE_DIR = Path.cwd() / "__pycache__"
DATA_DIR = Path.cwd() / "Data"

purge_cache = False  # Set to True to delete the cache directory

if purge_cache and CACHE_DIR.exists():
    print("Purging cache directory")
    for file in CACHE_DIR.iterdir():
        file.unlink()
    CACHE_DIR.rmdir()
    print("Cache directory purged")

if not CACHE_DIR.exists():
    print("Creating cache directory")
    CACHE_DIR.mkdir()
    print("Cache directory created")

if not DATA_DIR.exists():
    raise FileNotFoundError(
        "Data directory not found. Please reclone correctly the repository."
    )

# Running simulation

In [ ]:
if os.name == "nt":
    EXECUTABLE_PATH = Path.cwd() / "bin" / "xva.exe"
    build_command = "make windows RELEASE=TRUE"
else:
    EXECUTABLE_PATH = Path.cwd() / "bin" / "xva.out"
    build_command = "make linux RELEASE=TRUE"

use_gpu = False

available_gpu = use_gpu and torch.cuda.is_available()

In [ ]:
M0 = 10
M1 = 100
N = 10**3
T = 1

XVAS: Dict[str, float] = {"CVA": 1.4}

if not XVAS:
    raise ValueError("Please set the XVA parameters")

XVAS_str = ",".join([f"{key}={value}" for key, value in XVAS.items()])

In [ ]:
if not EXECUTABLE_PATH.exists():
    print("Building the executable")
    if os.system(build_command) != 0:
        raise RuntimeError("Failed to build the executable")

print("All set up!")

if available_gpu:
    print("GPU is available")

    if shutil.which("device-query") is not None:
        os.system("device-query")

    t1 = perf_counter()
    if os.system(f"{EXECUTABLE_PATH} {M0} {M1} {N} {T} {XVAS_str}"):
        raise RuntimeError("Failed to run the executable")
    print(f"Execution time: {perf_counter() - t1} seconds")

else:
    print("GPU is not available or not selected. Running on CPU.")
    t1 = perf_counter()
    if os.system(f"{EXECUTABLE_PATH} --cpu {M0} {M1} {N} {T} {XVAS_str}"):
        raise RuntimeError("Failed to run the executable")
    print(f"Execution time: {perf_counter() - t1} seconds")

print("Completed job")

In [ ]:
df = pd.read_csv("Data/results.csv", index_col=0)

df.head()

In [ ]:
plt.figure(figsize=(15, 5))
plt.title("XVA")
plt.xlabel("Time")
plt.ylabel("Value")

for col in df.columns:
    plt.plot(df.index, df[col], label=col)

plt.legend()
plt.show()